In [5]:
import os
HOME = os.getcwd()
print(HOME)



/media/marcin/5AD6135FD6133B29/Programowanie/Python/Magisterka/Projekt Badawczy/Github/Ticket-to-ride


In [1]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.190 🚀 Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 2060, 5924MiB)
Setup complete ✅ (12 CPUs, 15.5 GB RAM, 43.6/77.4 GB disk)


In [2]:
from ultralytics import YOLO
from IPython.display import display, Image

In [9]:
%cd {HOME}
!yolo task=detect mode=predict model=Roboflow_model/best.pt conf=0.25 source=dataset/age_of_steam/board save=True save_txt=True


/media/marcin/5AD6135FD6133B29/Programowanie/Python/Magisterka/Projekt Badawczy/Github/Ticket-to-ride
/bin/bash: /home/marcin/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Ultralytics YOLOv8.0.190 🚀 Python-3.11.3 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 2060, 5924MiB)
Model summary (fused): 168 layers, 11158866 parameters, 0 gradients, 28.6 GFLOPs

image 1/1 /media/marcin/5AD6135FD6133B29/Programowanie/Python/Magisterka/Projekt Badawczy/Github/Ticket-to-ride/dataset/age_of_steam/board/original.jpg: 480x640 1 10_0, 1 10_2, 1 10_4, 1 11_1, 1 12_2, 1 12_4, 1 13_4, 1 14_1, 1 14_3, 2 1_4s, 1 24_0, 2 2_0s, 3 2_1s, 3 2_2s, 1 2_3, 2 2_4s, 1 2_5, 4 3_0s, 1 3_1, 7 3_2s, 1 4_5, 1 6_1, 1 7_1, 1 9_0, 1 9_2, 28.3ms
Speed: 3.0ms preprocess, 28.3ms inference, 5.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels
💡 Learn more at https://docs.ultralytics.com/modes/predict

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import math

# Rozmiar planszy
szerokosc = 640
wysokosc = 640

# Liczba kolumn i wierszy w planszy
liczba_kolumn = 18
wiersze_w_kolumnach = [11, 10, 11, 10, 11, 10, 11, 10, 11, 10, 11,10,11,10,11,10,11,10]
wiersze = 11

# Rozmiar heksów
rozmiar_heksa = 20
odstep = rozmiar_heksa * 3**0.5 / 2  # Odległość między środkami heksów

# Tworzenie obrazu planszy
plansza = Image.new('RGB', (szerokosc, wysokosc), (255, 255, 255))
draw = ImageDraw.Draw(plansza)


tab = [[None for _ in range(liczba_kolumn)] for _ in range(liczba_wierszy)]

tab[5][7]  = "1B"
tab[9][5]  = "2B"
tab[10][0] = "3B"
tab[7][2]  = "4B"
tab[2][1]  = "5B"
tab[0][2]  = "6B"
tab[9][13] = "M"
tab[9][9]  = "1C"
tab[8][12] = "2C"
tab[3][13] = "3C"
tab[8][16] = "4C"
tab[5][17] = "5C"
tab[1][16] = "6C"


def rysuj_heks(x, y, rozmiar, kat_obrotu, tekst=None):
    x0, y0 = x + rozmiar * math.cos(kat_obrotu), y - rozmiar * math.sin(kat_obrotu)
    x1, y1 = x + rozmiar * math.cos(kat_obrotu + math.radians(60)), y - rozmiar * math.sin(kat_obrotu + math.radians(60))
    x2, y2 = x + rozmiar * math.cos(kat_obrotu + math.radians(120)), y - rozmiar * math.sin(kat_obrotu + math.radians(120))
    x3, y3 = x + rozmiar * math.cos(kat_obrotu + math.radians(180)), y - rozmiar * math.sin(kat_obrotu + math.radians(180))
    x4, y4 = x + rozmiar * math.cos(kat_obrotu + math.radians(240)), y - rozmiar * math.sin(kat_obrotu + math.radians(240))
    x5, y5 = x + rozmiar * math.cos(kat_obrotu + math.radians(300)), y - rozmiar * math.sin(kat_obrotu + math.radians(300))

    points = [x0, y0, x1, y1, x2, y2, x3, y3, x4, y4, x5, y5]
    draw.polygon(points, outline=(0, 0, 0))

    if tekst:
        srodek_x = x
        srodek_y = y
        # Ustaw czcionkę i rozmiar tekstu
        czcionka = ImageFont.load_default()
        # Ustaw kolor tekstu
        kolor_tekstu = (0, 0, 0)
        # Wstaw tekst w środku heksa
        text_width, text_height = draw.textsize(tekst, font=czcionka)
        text_x = srodek_x - text_width // 2
        text_y = srodek_y - text_height // 2
        draw.text((text_x, text_y), tekst, font=czcionka, fill=kolor_tekstu)

# Rysowanie planszy
x = rozmiar_heksa
y = rozmiar_heksa

for i, kolumna in enumerate(range(liczba_kolumn)):
    if i % 2 == 0:
        y = rozmiar_heksa
    else:
        y = rozmiar_heksa + rozmiar_heksa * 3**0.5/2
    for j, wiersze in enumerate(wiersze_w_kolumnach):
        if i < liczba_kolumn and j < wiersze:
            if tab[j][i] is not None:
                if i % 2 != 1 or j!=10:
                  rysuj_heks(x, y, rozmiar_heksa, 0, tekst=tab[j][i])
            else:
                if i % 2 != 1 or j!=10:
                  rysuj_heks(x, y, rozmiar_heksa, 0, tekst="")
        y += odstep * 2
    x += odstep * 2



# Zapisz planszę do pliku
plansza.save('plansza_hex.png')
obraz = Image.open('plansza_hex.png')
obraz2 = Image.open('/content/runs/detect/predict/IMG_20231012_210315_jpg.rf.6d3af08fd8a1420847552a273c3fb57b.jpg')

#Wyświetlanie
fig = plt.figure(figsize=(10, 8))
plt.imshow(obraz)
plt.axis('off')
plt.show()
fig1 = plt.figure(figsize=(10, 8))
plt.imshow(obraz2)
plt.axis('off')
plt.show()